<a href="https://colab.research.google.com/github/Prof-Luis1986/Tutoriales_flet/blob/main/VideoPlayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica: Reproductor de video con Flet + flet-video
1. Requisitos previos

* Python 3.9+ instalado.

* VS Code (o editor similar).

* Conexión a internet para descargar dependencias.

* Linux únicamente: instalar libmpv antes de usar video.

2. Crear carpeta y entorno virtual

macOS / Linux

In [ ]:
mkdir videoplayer && cd videoplayer
python3 -m venv .venv
source .venv/bin/activate


Windows (PowerShell)

In [ ]:
md videoplayer; cd videoplayer
py -m venv .venv
.\.venv\Scripts\Activate


3. Instalar dependencias

In [ ]:
pip install --upgrade pip
pip install flet
pip install flet-video

# Elaboremos el programa

1. Imports: traemos las librerías

In [ ]:
import flet as ft
import flet_video as fv

* flet (ft) es el framework de la interfaz.

* flet_video (fv) aporta el Video, VideoMedia, y PlaylistMode.

2. Punto de entrada de la app

In [ ]:
def main(page: ft.Page):
    ...
ft.app(target=main)

* main(page) es la función que Flet llama para construir la pantalla.

* ft.app(target=main) arranca la aplicación y le dice a Flet que use main.

3. Ajustes de la ventana y del layout (dentro de main)

In [ ]:
page.title = "VideoPlayer"
page.theme_mode = ft.ThemeMode.LIGHT
page.window.always_on_top = True
page.spacing = 10
page.horizontal_alignment = ft.CrossAxisAlignment.CENTER
page.padding = 10


* Título de la ventana, tema claro y ventana “siempre al frente”.

* Espaciado general y alineación horizontal al centro.

* padding=10 reduce bordes para que todo se vea compacto.

4. Datos de los videos (fuente de verdad)

In [ ]:
video_data = [
    {"title": "Video 1: Naturaleza", "url": "https://...mp4"},
    ...
]

* Es la lista con cada video: título + URL en mp4.

* Si algún día quieren cambiar videos, este es el lugar

5. Convertimos los datos en medios de reproducción

In [ ]:
playlist = [fv.VideoMedia(item["url"]) for item in video_data]


* Crea una playlist con objetos VideoMedia a partir de cada url.

* Esta lista es la que entiende el componente Video.

6. Estado: índice actual y título visible

In [ ]:
current_index = ft.Ref[int]()
current_index.current = 0

current_title = ft.Text(video_data[0]["title"], style="titleLarge", text_align=ft.TextAlign.CENTER)


7. SnackBar para mensajes rápidos

In [ ]:
snackbar = ft.SnackBar(content=ft.Text(""))

* El SnackBar se usa para avisos temporales: volumen, velocidad, etc.

* Se abre/cierra cuando lo necesitemos (más abajo lo verás en los sliders).

8. Creamos el reproductor de video

In [ ]:
video = fv.Video(
    expand=True,
    playlist=playlist,
    playlist_mode=fv.PlaylistMode.LOOP,
    aspect_ratio=16 / 9,
    volume=100,
    autoplay=False,
    muted=False,
    fill_color=ft.Colors.BLUE_400,
    filter_quality=ft.FilterQuality.HIGH,
    on_loaded=lambda e: print("Video cargado correctamente."),
)


Props clave:

* playlist=playlist: la lista del Paso 5.

* playlist_mode=...LOOP: al terminar, vuelve al primero (repite la lista).

* aspect_ratio=16/9: proporción estándar del reproductor.

* volume=100, autoplay=False, muted=False: estado inicial.

* on_loaded=...: cuando el video está listo, imprime un mensaje en la consola.

expand=True hace que el video use el espacio disponible.

9. Handlers básicos de reproducción

In [ ]:
def handle_play(e):
    video.play()

def handle_pause(e):
    video.pause()


* Dos funciones muy directas: Play y Pause.

* Se conectarán a botones para que el usuario controle el video.

10. Handlers para navegar la playlist

In [ ]:
def handle_next(e):
    next_index = (current_index.current + 1) % len(video_data)
    current_index.current = next_index
    video.next()
    current_title.value = video_data[next_index]["title"]
    page.update()

def handle_previous(e):
    prev_index = (current_index.current - 1) % len(video_data)
    current_index.current = prev_index
    video.previous()
    current_title.value = video_data[prev_index]["title"]
    page.update()


* Calculan el siguiente o anterior índice usando módulo (%) para ciclar.

* Llaman video.next() / video.previous() para cambiar la pista.

* Actualizan el título visible y hacen page.update() para refrescar la pantalla.

11. Handler del volumen (slider)

In [ ]:
def handle_volume_change(e):
    video.volume = int(e.control.value)
    video.pause()
    video.play()
    snackbar.content.value = f"🔊 Volumen: {video.volume}%"
    page.snack_bar = snackbar
    page.snack_bar.open = True
    page.update()


* Conviertes el valor del slider a entero y lo asignas a video.volume.

* Pausa y reproduce para aplicar el cambio de forma consistente.

* Muestra un SnackBar con el porcentaje actual y actualiza la página.

12. Handler de la velocidad de reproducción (slider)

In [ ]:
def handle_playback_rate_change(e):
    video.playback_rate = float(e.control.value)
    video.pause()
    video.play()
    snackbar.content.value = f"⏩ Velocidad: {video.playback_rate}x"
    page.snack_bar = snackbar
    page.snack_bar.open = True
    page.update()


* Convierte el valor a float y lo asigna a video.playback_rate (0.5x a 2x).

* Igual que el volumen: pausa/reproduce, muestra SnackBar y actualiza.

13. Función utilitaria para crear botones redondeados

In [ ]:
def rounded_button(text, handler):
    return ft.ElevatedButton(
        text,
        on_click=handler,
        style=ft.ButtonStyle(
            shape=ft.RoundedRectangleBorder(radius=20),
            padding=20,
        )
    )


* Evita repetir estilos en cada botón.

* Le pasas el texto y el handler; regresa un botón ya estilizado.

14. Fila de controles (los 4 botones)

In [ ]:
controls = ft.Row(
    wrap=True,
    alignment=ft.MainAxisAlignment.CENTER,
    spacing=10,
    controls=[
        rounded_button("Play", handle_play),
        rounded_button("Pause", handle_pause),
        rounded_button("Previous", handle_previous),
        rounded_button("Next", handle_next),
    ]
)


* Crea una fila con Play, Pause, Previous y Next.

* wrap=True permite que salten de línea si la pantalla es pequeña.

* alignment=CENTER los centra horizontalmente.

15. Sliders: volumen y velocidad

In [ ]:
sliders = ft.Column([
    ft.Slider(
        min=0, value=100, max=100, divisions=10, width=400,
        label="Volumen = {value}%",
        on_change=handle_volume_change
    ),
    ft.Slider(
        min=0.5, value=1, max=2, divisions=6, width=400,
        label="Velocidad = {value}x",
        on_change=handle_playback_rate_change
    )
])


Dos sliders apilados en una columna:

* Volumen (0–100) con división en pasos de 10.

* Velocidad (0.5x–2x) con 6 divisiones.

Cada slider llama a su handler al cambiar.

16. Composición visual final y fondo degradado

In [ ]:
page.add(
    ft.Container(
        expand=True,
        padding=0,
        margin=0,
        gradient=ft.LinearGradient(
            begin=ft.alignment.top_center,
            end=ft.alignment.bottom_center,
            colors=[ft.Colors.LIGHT_BLUE_100, ft.Colors.BLUE_700],
        ),
        content=ft.Column(
            alignment=ft.MainAxisAlignment.CENTER,
            horizontal_alignment=ft.CrossAxisAlignment.CENTER,
            spacing=25,
            controls=[
                current_title,
                video,
                controls,
                sliders
            ]
        )
    )
)


* Se agrega a la página un Container que ocupa todo (expand=True) y tiene un fondo degradado vertical.

* Dentro, una Column centrada que acomoda:

1. el título actual,

2. el reproductor,

3. los botones,

4. los sliders.

17. Arranque (ya visto, pero clave)

In [ ]:
ft.app(target=main)


* Llama a main y lanza la aplicación.

* Para probar, ejecuten el archivo.

18. ¿Qué deben observar al ejecutar?

* Se abre una ventana “VideoPlayer”.

* El primer video está listo; al Play, comienza la reproducción.

* Next/Previous cambian de video y actualizan el título.

* Al mover Volumen o Velocidad, aparece un SnackBar con el valor.

* Al terminar el último video, la lista vuelve a empezar (modo LOOP).

19. Código completo

In [ ]:
import flet as ft
import flet_video as fv


def main(page: ft.Page):
    page.title = "VideoPlayer"
    page.theme_mode = ft.ThemeMode.LIGHT
    page.window.always_on_top = True
    page.spacing = 10
    page.horizontal_alignment = ft.CrossAxisAlignment.CENTER
    page.padding =10  # Elimina bordes blancos

    video_data = [
        {"title": "Video 1: Naturaleza", "url": "https://user-images.githubusercontent.com/28951144/229373720-14d69157-1a56-4a78-a2f4-d7a134d7c3e9.mp4"},
        {"title": "Video 2: Ciudad", "url": "https://user-images.githubusercontent.com/28951144/229373718-86ce5e1d-d195-45d5-baa6-ef94041d0b90.mp4"},
        {"title": "Video 3: Espacio", "url": "https://user-images.githubusercontent.com/28951144/229373716-76da0a4e-225a-44e4-9ee7-3e9006dbc3e3.mp4"},
        {"title": "Video 4: Animación", "url": "https://user-images.githubusercontent.com/28951144/229373695-22f88f13-d18f-4288-9bf1-c3e078d83722.mp4"},
        {"title": "Video 5: Música", "url": "https://user-images.githubusercontent.com/28951144/229373709-603a7a89-2105-4e1b-a5a5-a6c3567c9a59.mp4"},
    ]

    playlist = [fv.VideoMedia(item["url"]) for item in video_data]

    current_index = ft.Ref[int]()
    current_index.current = 0

    current_title = ft.Text(video_data[0]["title"], style="titleLarge", text_align=ft.TextAlign.CENTER)

    snackbar = ft.SnackBar(content=ft.Text(""))

    video = fv.Video(
        expand=True,
        playlist=playlist,
        playlist_mode=fv.PlaylistMode.LOOP,
        aspect_ratio=16 / 9,
        volume=100,
        autoplay=False,
        muted=False,
        fill_color=ft.Colors.BLUE_400,
        filter_quality=ft.FilterQuality.HIGH,
        on_loaded=lambda e: print("Video cargado correctamente."),
    )

    def handle_play(e):
        video.play()

    def handle_pause(e):
        video.pause()

    def handle_next(e):
        next_index = (current_index.current + 1) % len(video_data)
        current_index.current = next_index
        video.next()
        current_title.value = video_data[next_index]["title"]
        page.update()

    def handle_previous(e):
        prev_index = (current_index.current - 1) % len(video_data)
        current_index.current = prev_index
        video.previous()
        current_title.value = video_data[prev_index]["title"]
        page.update()

    def handle_volume_change(e):
        video.volume = int(e.control.value)
        video.pause()
        video.play()
        snackbar.content.value = f"🔊 Volumen: {video.volume}%"
        page.snack_bar = snackbar
        page.snack_bar.open = True
        page.update()

    def handle_playback_rate_change(e):
        video.playback_rate = float(e.control.value)
        video.pause()
        video.play()
        snackbar.content.value = f"⏩ Velocidad: {video.playback_rate}x"
        page.snack_bar = snackbar
        page.snack_bar.open = True
        page.update()

    # UI: controles redondeados
    def rounded_button(text, handler):
        return ft.ElevatedButton(
            text,
            on_click=handler,
            style=ft.ButtonStyle(
                shape=ft.RoundedRectangleBorder(radius=20),
                padding=20,
            )
        )

    controls = ft.Row(
        wrap=True,
        alignment=ft.MainAxisAlignment.CENTER,
        spacing=10,
        controls=[
            rounded_button("Play", handle_play),
            rounded_button("Pause", handle_pause),
            rounded_button("Previous", handle_previous),
            rounded_button("Next", handle_next),
        ]
    )

    sliders = ft.Column([
        ft.Slider(
            min=0,
            value=100,
            max=100,
            divisions=10,
            width=400,
            label="Volumen = {value}%",
            on_change=handle_volume_change
        ),
        ft.Slider(
            min=0.5,
            value=1,
            max=2,
            divisions=6,
            width=400,
            label="Velocidad = {value}x",
            on_change=handle_playback_rate_change
        )
    ])

    page.add(
        ft.Container(
            expand=True,
            padding=0,
            margin=0,
            gradient=ft.LinearGradient(
                begin=ft.alignment.top_center,
                end=ft.alignment.bottom_center,
                colors=[ft.Colors.LIGHT_BLUE_100, ft.Colors.BLUE_700],
            ),
            content=ft.Column(
                alignment=ft.MainAxisAlignment.CENTER,
                horizontal_alignment=ft.CrossAxisAlignment.CENTER,
                spacing=25,
                controls=[
                    current_title,
                    video,
                    controls,
                    sliders
                ]
            )
        )
    )


ft.app(target=main)


#20 Responde en tu google sites.

1. ¿Para qué sirven las importaciones flet y flet_video en este proyecto?

2. ¿Qué papel cumple la función main(page: ft.Page) y cómo se inicia la app?

3. ¿Qué efecto tienen page.theme_mode, page.window.always_on_top y page.padding en la ventana?

4. ¿Qué estructura tiene cada elemento de video_data y para qué se usa esa lista?

5. ¿Qué hace la comprensión de listas que construye playlist = [fv.VideoMedia(item["url"]) for item in video_data]?

6. ¿Para qué se utiliza ft.Ref[int]() en current_index y por qué no basta con una variable normal?

7. ¿Qué muestra current_title y en qué momentos del código se actualiza su valor?

8. ¿Qué parámetros de fv.Video controlan el comportamiento del reproductor (como playlist, playlist_mode, aspect_ratio, volume, autoplay, muted)?

9. ¿Qué función cumple el callback on_loaded del reproductor?

10. ¿Cómo calculan handle_next y handle_previous el índice siguiente/anterior y por qué usan el operador %?

11. ¿Por qué en los handlers de volumen y velocidad se hace video.pause() seguido de video.play()?

12. ¿Qué información muestra el SnackBar y cómo se abre desde el código?

13. ¿Qué ventaja aporta la función rounded_button al crear los cuatro botones de control?

14. ¿Qué rol tienen los atributos de los Slider (min, max, divisions, label, on_change) y cómo se vinculan con sus handlers?

15. ¿Cómo se arma el layout final dentro del Container con fondo degradado y qué orden siguen los controles en la Column?